# Loading Data
**For more examples of what Kosh can do visit [GitHub Examples](https://github.com/LLNL/kosh/tree/stable/examples).**

In [ ]:
from numbers import Number
from collections import defaultdict

import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import kosh
import math
import statistics
import numpy as np
import os
import sys

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

if sys.argv[1] == '-f':  # Running as notebook
    out_path = ''
    %matplotlib notebook
else:
    out_path = sys.argv[1]  # Running as script

# Ensembles Initialization
database = os.path.join(out_path, 'ensembles_output.sqlite')
print(database)
datastore = kosh.connect(database)
print("Kosh is ready!")

# Printing Attributes and Features
test_rec = list(datastore.find())[1]
print('Attributes:')
print('\t',test_rec.list_attributes())
print('\n')
print('Features Sets:')
print('\t',test_rec.list_features())
time=test_rec['physics_cycle_series/time'][:]
image_path = os.path.join(out_path, 'lstm-ball-bounce/images')
os.makedirs(image_path, exist_ok=True)

# Loading Data

Create a scaled Rank 3 Tensor for each of 60% Training, 20% Validation, and 20% Test Data.

(# of Datasets, # of Time Steps per Dataset, # of Features per Time Step)

**Cyclic Data**

If the data is cyclical, the data can be split within each dataset since it has seen all the "patterns" of the cycle.

**Acyclic Data**

However, for acyclic data splitting it must be done at the dataset level instead of within the dataset since the model will have not seen the whole "pattern".

In [ ]:
# Initialize Arrays
X_train=np.array([])
X_val=np.array([])
X_test=np.array([])

data_type = 'acyclic' # 'cyclic'

if data_type == 'cyclic':

    for i, dataset in enumerate(datastore.find(load_type='dictionary')): # Each record is now a dataset

            print(f"----------------------Dataset #{i}: ID: {dataset['id']}----------------------")
            if dataset['id']=='mean':
                continue

            x_pos = dataset['curve_sets']['physics_cycle_series']['dependent']['x_pos']['value'][:]
            y_pos = dataset['curve_sets']['physics_cycle_series']['dependent']['y_pos']['value'][:]
            z_pos = dataset['curve_sets']['physics_cycle_series']['dependent']['z_pos']['value'][:]

            # Current dataset matrix for features
            X = pd.DataFrame([x_pos, y_pos, z_pos]).transpose()
            X.columns=['x_pos','y_pos','z_pos']

            # Splitting into train, validation, and test but since this is a time-series, it can't be shuffled
            # Train: 60%, Validation: 20%, Test: 20%
            X_train_temp, X_test_temp = train_test_split(X, test_size = 0.2, shuffle=False)
            X_train_temp, X_val_temp = train_test_split(X_train_temp, test_size = 0.25, shuffle=False)

            print('All Data:\n',X,'\n')
            print('Training Data\n', X_train_temp, '\n')
            print('Validation Data\n', X_val_temp, '\n')
            print('Test Data\n', X_test_temp, '\n')

            # Concatenating the tensor
            if X_train.size==0:
                X_train = np.array([X_train_temp.values])
                X_val = np.array([X_val_temp.values ])
                X_test = np.array([X_test_temp.values ])
            else:
                X_train = np.vstack((X_train,[X_train_temp.values]))
                X_val =  np.vstack((X_val,[X_val_temp.values]))
                X_test =  np.vstack((X_test,[X_test_temp.values]))            

    # Plotting for cylical data
    fig, ax = plt.subplots(nrows=3,sharex=True)
    fig.suptitle('Example of Train, Val, and Test split')
    time_train = time[:len(X_train_temp['x_pos'])]
    time_val = time[len(X_train_temp['x_pos']):len(X_train_temp['x_pos'])+len(X_val_temp['x_pos'])]
    time_test = time[len(X_train_temp['x_pos'])+len(X_val_temp['x_pos']):]

    ax[0].plot(time,x_pos, label='Original')
    ax[0].plot(time_train ,X_train_temp['x_pos'], label='Train')
    ax[0].plot(time_val,X_val_temp['x_pos'], label='Validation')
    ax[0].plot(time_test,X_test_temp['x_pos'], label='Test')
    ax[0].legend(fontsize='xx-small')
    ax[0].set_title('x_pos')

    ax[1].plot(time,y_pos, label='Original')
    ax[1].plot(time_train ,X_train_temp['y_pos'], label='Train')
    ax[1].plot(time_val,X_val_temp['y_pos'], label='Validation')
    ax[1].plot(time_test,X_test_temp['y_pos'], label='Test')
    ax[1].legend(fontsize='xx-small')
    ax[1].set_title('y_pos')

    ax[2].plot(time,z_pos, label='Original')
    ax[2].plot(time_train ,X_train_temp['z_pos'], label='Train')
    ax[2].plot(time_val,X_val_temp['z_pos'], label='Validation')
    ax[2].plot(time_test,X_test_temp['z_pos'], label='Test')
    ax[2].legend(fontsize='xx-small')
    ax[2].set_title('z_pos')

    plt.tight_layout()
    fig.savefig(os.path.join(image_path, 'cyclic_example_split.png'))

elif data_type == 'acyclic':

    num_datasets = len(list(datastore.find(load_type='dictionary'))) -1 # Subtract the 'mean' dataset from the other notebook
    train_datasets = round(num_datasets*.6)
    val_datasets = round(num_datasets*.2)
    test_datasets = round(num_datasets*.2)    
    print('Number of train, val, and test datasets:', train_datasets, val_datasets, test_datasets)

    for i, dataset in enumerate(datastore.find(load_type='dictionary')): # Each record is now a dataset

            print(f"----------------------Dataset #{i}: ID: {dataset['id']}----------------------")
            if dataset['id']=='mean':
                continue

            x_pos = dataset['curve_sets']['physics_cycle_series']['dependent']['x_pos']['value'][:]
            y_pos = dataset['curve_sets']['physics_cycle_series']['dependent']['y_pos']['value'][:]
            z_pos = dataset['curve_sets']['physics_cycle_series']['dependent']['z_pos']['value'][:]

            # Current dataset matrix for features
            X = pd.DataFrame([x_pos, y_pos, z_pos]).transpose()
            X.columns=['x_pos','y_pos','z_pos']

            # Concatenating the tensor
            if i<train_datasets:
                if X_train.size==0:
                    X_train = np.array([X.values])
                else:
                    X_train = np.vstack((X_train,[X.values])) 
            elif i<train_datasets+val_datasets:
                if X_val.size==0:
                    X_val = np.array([X.values ])
                else:
                    X_val =  np.vstack((X_val,[X.values]))
            else:
                if X_test.size==0:
                    X_test = np.array([X.values ])
                else:
                    X_test =  np.vstack((X_test,[X.values]))   

    # Plotting for acyclical data
    fig, ax = plt.subplots(nrows=3,sharex=True)
    fig.suptitle('Example of Train, Val, and Test split')

    ax[0].plot(time,X_train[0,:,0], label='Train')
    ax[0].plot(time,X_val[0,:,0], label='Validation')
    ax[0].plot(time,X_test[0,:,0], label='Test')
    ax[0].legend(fontsize='xx-small')
    ax[0].set_title('x_pos')

    ax[1].plot(time ,X_train[0,:,1], label='Train')
    ax[1].plot(time,X_val[0,:,1], label='Validation')
    ax[1].plot(time,X_test[0,:,1], label='Test')
    ax[1].legend(fontsize='xx-small')
    ax[1].set_title('y_pos')

    ax[2].plot(time ,X_train[0,:,2], label='Train')
    ax[2].plot(time,X_val[0,:,2], label='Validation')
    ax[2].plot(time,X_test[0,:,2], label='Test')
    ax[2].legend(fontsize='xx-small')
    ax[2].set_title('z_pos')

    plt.tight_layout()
    fig.savefig(os.path.join(image_path, 'acyclic_example_split.png'))

## Scaling the data
Scaling the data so that all the features are around the same magnitude helps the model converge faster due to how the optimizers update the weights.

In [ ]:
num_time_steps, num_features = X.shape
print('Each whole dataset:',X.shape)
print('\tNumber of Time Steps in Each whole dataset:', num_time_steps) 
print('\tNumber of Features per Time Step:', num_features) 
print('\n')

# Scaler
scaler = StandardScaler()

##############
# Train Data #
##############
num_datasets, num_time_steps, num_features = X_train.shape
print('X_train:',X_train.shape)
print('\tNumber of Datasets:', num_datasets) 
print('\tNumber of Time Steps per Dataset:', num_time_steps) 
print('\tNumber of Features per Time Step:', num_features) 
print('\n')

# Reshape each feature for all datasets into one long feature for scaling
X_train = np.reshape(X_train, newshape=(-1, num_features))
X_train = scaler.fit_transform(X_train) # Fit AND transform only for train data

# Reshape each long feature back into their own dataset
X_train_scaled = np.reshape(X_train, newshape=(num_datasets, num_time_steps, num_features))


###################
# Validation Data #
###################
num_datasets, num_time_steps, num_features = X_val.shape
print('X_val:',X_val.shape)
print('\tNumber of Datasets:', num_datasets) 
print('\tNumber of Time Steps per Dataset:', num_time_steps) 
print('\tNumber of Features per Time Step:', num_features) 
print('\n')

# Reshape each feature for all datasets into one long feature for scaling
X_val = np.reshape(X_val, newshape=(-1, num_features))
X_val = scaler.transform(X_val)  # Transform ONLY for validation data

# Reshape each long feature back into their own dataset
X_val_scaled = np.reshape(X_val, newshape=(num_datasets, num_time_steps, num_features))


#############
# Test Data #
#############
num_datasets, num_time_steps, num_features = X_test.shape
print('X_test:',X_test.shape)
print('\tNumber of Datasets:', num_datasets) 
print('\tNumber of Time Steps per Dataset:', num_time_steps) 
print('\tNumber of Features per Time Step:', num_features) 
print('\n')

# Reshape each feature for all datasets into one long feature for scaling
X_test = np.reshape(X_test, newshape=(-1, num_features))
X_test = scaler.transform(X_test)  # Transform ONLY for test data

# Reshape each long feature back into their own dataset
X_test_scaled = np.reshape(X_test, newshape=(num_datasets, num_time_steps, num_features))

# Splitting Data into batches for LSTM

Each dataset needs to be split into batches for LSTM to process. These batches are created such that the next time step is the y label. The size of the batch can be adjusted with `window`. Each batch increments a time step.

The 0th batch of a single dataset with `window=3`:

$$
X_{train_{batch=0}} = 
\begin{bmatrix}
    [xpos_{t=0} & ypos_{t=0} & zpos_{t=0}] \\
    [xpos_{t=1} & ypos_{t=1} & zpos_{t=1}] \\
    [xpos_{t=2} & ypos_{t=2} & zpos_{t=2}]
\end{bmatrix}
\\
y_{train_{batch=0}} = 
\begin{bmatrix}
    [xpos_{t=3} & ypos_{t=3} & zpos_{t=3}]
\end{bmatrix}
$$

The 1st batch of a single dataset with `window=3`:

$$
X_{train_{batch=1}} = 
\begin{bmatrix}
    [xpos_{t=1} & ypos_{t=1} & zpos_{t=1}] \\
    [xpos_{t=2} & ypos_{t=2} & zpos_{t=2}] \\
    [xpos_{t=3} & ypos_{t=3} & zpos_{t=3}]
\end{bmatrix}
\\
y_{train_{batch=1}} = 
\begin{bmatrix}
    [xpos_{t=4} & ypos_{t=4} & zpos_{t=4}]
\end{bmatrix}
$$

etc...

However this will cause the tensor to become Rank 4 since we added the dimension of batches. We will need to reshape the X tensor to Rank 3 so that the LSTM can process it and since the y tensor only has 1 timestep of prediction, we will need to reshape it to Rank 2. If it was more than one timestep, we would turn it into a Rank 3 like X.

X and y from Rank 4:

(# of Datasets, # of Batches per Dataset, # of Time Steps per Batch, # of Features per Time Step)

X to Rank 3:

(# of Total Batches = Batches * Datasets, # of Time Steps per Batch, # of Features per Time Step)

y to Rank 2:

(# of Total Batches = Batches * Datasets, # of Features per Single Time Step)

In [ ]:
def lstm_split(X_train,window,X_val,X_test):

    # Train data for lstm
    num_datasets, num_time_steps, num_features = X_train.shape
    total_batches_train = num_time_steps-window
    X_train_lstm = np.array([])
    y_train_lstm = np.array([])

    for dataset in range(num_datasets):
        print(f"----------------------{dataset}----------------------")
        X_train_dataset = X_train[dataset]

        X_train_lstm_temp =[]
        y_train_lstm_temp = []

        # Train data for lstm
        for i in range(total_batches_train):
            X_train_lstm_temp.append(X_train_dataset[i:window+i])
            y_train_lstm_temp.append([X_train_dataset[window+i]])

        X_train_lstm_temp = np.array(X_train_lstm_temp)
        y_train_lstm_temp = np.array(y_train_lstm_temp)

        
        if X_train_lstm.size==0:
            X_train_lstm = np.array([X_train_lstm_temp])
            y_train_lstm = np.array([y_train_lstm_temp])
            
        else:
            X_train_lstm = np.vstack((X_train_lstm,[X_train_lstm_temp]))
            y_train_lstm = np.vstack((y_train_lstm,[y_train_lstm_temp]))

    # Val data for lstm
    num_datasets, num_time_steps, num_features = X_val.shape
    total_batches_val = num_time_steps-window
    X_val_lstm = np.array([])
    y_val_lstm = np.array([])

    for dataset in range(num_datasets):
        print(f"----------------------{dataset}----------------------")
        X_val_dataset = X_val[dataset]

        X_val_lstm_temp =[]
        y_val_lstm_temp = []

        # Val data for lstm
        for i in range(total_batches_val):
            X_val_lstm_temp.append(X_val_dataset[i:window+i])
            y_val_lstm_temp.append([X_val_dataset[window+i]])

        X_val_lstm_temp = np.array(X_val_lstm_temp)
        y_val_lstm_temp = np.array(y_val_lstm_temp)

        
        if X_val_lstm.size==0:
            X_val_lstm = np.array([X_val_lstm_temp])
            y_val_lstm =np.array([ y_val_lstm_temp])
            
        else:
            X_val_lstm = np.vstack((X_val_lstm,[X_val_lstm_temp]))
            y_val_lstm = np.vstack((y_val_lstm,[y_val_lstm_temp]))
            

    # Test data for lstm
    num_datasets, num_time_steps, num_features = X_test.shape
    total_batches_test = num_time_steps-window
    X_test_lstm = np.array([])
    y_test_lstm = np.array([])

    for dataset in range(num_datasets):
        print(f"----------------------{dataset}----------------------")
        X_test_dataset = X_test[dataset]

        X_test_lstm_temp =[]
        y_test_lstm_temp = []

        # Test data for lstm
        for i in range(total_batches_test):
            X_test_lstm_temp.append(X_test_dataset[i:window+i])
            y_test_lstm_temp.append([X_test_dataset[window+i]])

        X_test_lstm_temp = np.array(X_test_lstm_temp)
        y_test_lstm_temp = np.array(y_test_lstm_temp)

        
        if X_test_lstm.size==0:
            X_test_lstm = np.array([X_test_lstm_temp])
            y_test_lstm =np.array([ y_test_lstm_temp])
        else:
            X_test_lstm = np.vstack((X_test_lstm,[X_test_lstm_temp]))
            y_test_lstm = np.vstack((y_test_lstm,[y_test_lstm_temp]))
            
    ###################################
    # Only one dataset for prediction #
    ###################################
    # Last dataset so we can compare to train, val, and test loop
    X_train_lstm_for_predict = np.array([X_train_scaled[0]])
    X_val_lstm_for_predict  = np.array([X_val_scaled[0]])
    X_test_lstm_for_predict = np.array([X_test_scaled[0]])

    return (X_train_lstm, y_train_lstm,
            X_val_lstm, y_val_lstm,
            X_test_lstm, y_test_lstm,
            X_train_lstm_for_predict,
            X_val_lstm_for_predict,
            X_test_lstm_for_predict)

window = 30  # This also determines how many data points are required to predict a new time series

# All features
(X_train_lstm, y_train_lstm,
 X_val_lstm, y_val_lstm,
 X_test_lstm, y_test_lstm,
 X_train_lstm_for_predict,
 X_val_lstm_for_predict,
 X_test_lstm_for_predict) = lstm_split(X_train_scaled, window, X_val_scaled, X_test_scaled)

print('First 3 batches of a dataset')
print('X_train LSTM: \n',X_train_lstm[0,:3])
print('y_train LSTM: \n',y_train_lstm[0,:3])

print('---------------Data grouped by datasets for LSTM------------------------------')
print('\n')
print('X_train LSTM: ', X_train_lstm.shape)
print('\tNumber of Datasets:', X_train_lstm.shape[0]) 
print('\tNumber of Batches per Dataset:', X_train_lstm.shape[1]) 
print('\tNumber of Time Steps per Batch:', X_train_lstm.shape[2]) 
print('\tNumber of Features per Time Step:', X_train_lstm.shape[3]) 
print('\n')
print('y_train LSTM: ',y_train_lstm.shape)
print('\tNumber of Datasets:', y_train_lstm.shape[0]) 
print('\tNumber of Batches per Dataset:', y_train_lstm.shape[1]) 
print('\tNumber of Time Steps per Batch:', y_train_lstm.shape[2]) 
print('\tNumber of Features per Time Step:', y_train_lstm.shape[3]) 
print('\n')
print('X_val LSTM: ',X_val_lstm.shape)
print('\tNumber of Datasets:', X_val_lstm.shape[0]) 
print('\tNumber of Batches per Dataset:', X_val_lstm.shape[1]) 
print('\tNumber of Time Steps per Batch:', X_val_lstm.shape[2]) 
print('\tNumber of Features per Time Step:', X_val_lstm.shape[3]) 
print('\n')
print('y_val LSTM: ',y_val_lstm.shape)
print('\tNumber of Datasets:', y_val_lstm.shape[0]) 
print('\tNumber of Batches per Dataset:', y_val_lstm.shape[1]) 
print('\tNumber of Time Steps per Batch:', y_val_lstm.shape[2]) 
print('\tNumber of Features per Time Step:', y_val_lstm.shape[3]) 
print('\n')
print('X_test LSTM: ',X_test_lstm.shape)
print('\tNumber of Datasets:', X_test_lstm.shape[0]) 
print('\tNumber of Batches per Dataset:', X_test_lstm.shape[1]) 
print('\tNumber of Time Steps per Batch:', X_test_lstm.shape[2]) 
print('\tNumber of Features per Time Step:', X_test_lstm.shape[3]) 
print('\n')
print('y_test LSTM: ',y_test_lstm.shape)
print('\tNumber of Datasets:', y_test_lstm.shape[0]) 
print('\tNumber of Batches per Dataset:', y_test_lstm.shape[1]) 
print('\tNumber of Time Steps per Batch:', y_test_lstm.shape[2]) 
print('\tNumber of Features per Time Step:', y_test_lstm.shape[3]) 
print('\n')
print('\n')

print('---------------Single dataset for LSTM Prediction------------------------------')
print('\n')
print('X_train Whole:',X_train_lstm_for_predict.shape)
print('\tNumber of Batches per Single Dataset:', X_train_lstm_for_predict.shape[0]) 
print('\tNumber of Time Steps per Batch:', X_train_lstm_for_predict.shape[1]) 
print('\tNumber of Features per Time Step:', X_train_lstm_for_predict.shape[2]) 
print('\n')
print('X_val Whole:',X_val_lstm_for_predict.shape)
print('\tNumber of Batches per Single Dataset:', X_val_lstm_for_predict.shape[0]) 
print('\tNumber of Time Steps per Batch:', X_val_lstm_for_predict.shape[1]) 
print('\tNumber of Features per Time Step:', X_val_lstm_for_predict.shape[2]) 
print('\n')
print('X_test Whole:',X_test_lstm_for_predict.shape)
print('\tNumber of Batches per Single Dataset:', X_test_lstm_for_predict.shape[0]) 
print('\tNumber of Time Steps per Batch:', X_test_lstm_for_predict.shape[1]) 
print('\tNumber of Features per Time Step:', X_test_lstm_for_predict.shape[2]) 
print('\n')
print('\n')

print('---------------Reshape data for LSTM which takes 3D data------------------------------')
print('\n')

num_datasets, num_batches, num_time_steps, num_features = X_train_lstm.shape
X_train_lstm = np.reshape(X_train_lstm, newshape=(num_batches*num_datasets, num_time_steps, num_features))
num_datasets, num_batches, num_time_steps, num_features = y_train_lstm.shape
y_train_lstm = np.reshape(y_train_lstm, newshape=(num_batches*num_datasets, num_features))

print('X_train LSTM Reshaped: ', X_train_lstm.shape)
print('\tNumber of Total Batches = Datasets * Batches:', X_train_lstm.shape[0]) 
print('\tNumber of Time Steps per Batch:', X_train_lstm.shape[1]) 
print('\tNumber of Features per Time Step:', X_train_lstm.shape[2]) 
print('\n')
print('y_train LSTM Reshaped: ',y_train_lstm.shape)
print('\tNumber of Total Batches = Datasets * Batches:', y_train_lstm.shape[0]) 
print('\tNumber of Features per Single Time Step:', y_train_lstm.shape[1]) 
print('\n')

num_datasets, num_batches, num_time_steps, num_features_val = X_val_lstm.shape
X_val_lstm = np.reshape(X_val_lstm, newshape=(num_batches*num_datasets, num_time_steps, num_features_val))
num_datasets, num_batches, num_time_steps, num_features = y_val_lstm.shape
y_val_lstm = np.reshape(y_val_lstm, newshape=(num_batches*num_datasets, num_features))

print('X_val LSTM Reshaped: ', X_val_lstm.shape)
print('\tNumber of Total Batches = Datasets * Batches:', X_val_lstm.shape[0]) 
print('\tNumber of Time Steps per Batch:', X_val_lstm.shape[1]) 
print('\tNumber of Features per Time Step:', X_val_lstm.shape[2]) 
print('\n')
print('y_val LSTM Reshaped: ',y_val_lstm.shape)
print('\tNumber of Total Batches = Datasets * Batches:', y_val_lstm.shape[0]) 
print('\tNumber of Features per Single Time Step:', y_val_lstm.shape[1]) 
print('\n')

num_datasets, num_batches, num_time_steps, num_features_test = X_test_lstm.shape
X_test_lstm = np.reshape(X_test_lstm, newshape=(num_batches*num_datasets, num_time_steps, num_features_test))
num_datasets, num_batches, num_time_steps, num_features = y_test_lstm.shape
y_test_lstm = np.reshape(y_test_lstm, newshape=(num_batches*num_datasets, num_features))

print('X_test LSTM Reshaped: ', X_test_lstm.shape)
print('\tNumber of Total Batches = Datasets * Batches:', X_test_lstm.shape[0]) 
print('\tNumber of Time Steps per Batch:', X_test_lstm.shape[1]) 
print('\tNumber of Features per Time Step:', X_test_lstm.shape[2]) 
print('\n')
print('y_test LSTM Reshaped: ',y_test_lstm.shape)
print('\tNumber of Total Batches = Datasets * Batches:', y_test_lstm.shape[0]) 
print('\tNumber of Features per Single Time Step:', y_test_lstm.shape[1]) 
print('\n')
print('\n')

print('First 3 batches of a dataset Reshaped')
print('X_train LSTM: \n',X_train_lstm[:3])
print('y_train LSTM: \n',y_train_lstm[:3])
print('\n')
print('\n')

print("Double check we are grabbing the same dataset for comparison later on")
print(scaler.inverse_transform(X_train_scaled[0])[:5])
print(scaler.inverse_transform(X_train_lstm_for_predict[0])[:5])

# Train the Model
Use tensorflow to create the model and add layers as needed. The `tf.keras.layers.LSTM()` should only have `return_sequences=True` if it is not the last LSTM layer. If it is the last LSTM layer `return_sequences=False` so that it only returns a single next time step.

In [ ]:
###################
# Build the model #
###################

lstm_model = tf.keras.models.Sequential([
    # return_sequences=False on last lstm layer so only one time step is returned
    tf.keras.layers.LSTM(64,return_sequences=True), 
    tf.keras.layers.LSTM(32,return_sequences=True),
    tf.keras.layers.LSTM(16,return_sequences=True),
    tf.keras.layers.LSTM(8,return_sequences=False),
    tf.keras.layers.Dense(num_features)    # number of features
])


lstm_model.compile(loss='mse', optimizer='adam')



###################
# Train the model #
###################
# Early stopping just in case the model doesn't improve so we don't have to wait for all the epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, 
                                                  restore_best_weights=True)

# Train model
history = lstm_model.fit(X_train_lstm, y_train_lstm, 
                         epochs=100, # default epochs = 1
                         validation_data = (X_val_lstm, y_val_lstm),
                         callbacks=[early_stopping]
                        )

###########################
# Plot the learning curve #
###########################

fig, ax = plt.subplots()
ax.plot(history.history['loss'], label='Train')
ax.plot(history.history['val_loss'], label='Val')
ax.set_xlabel('epochs')
ax.set_ylabel('MSE')
ax.set_title('Train and Validation Loss')
ax.legend()

fig.savefig(os.path.join(image_path, 'learning_curve.png'))

######################
# Evaluate the model #
######################
print(lstm_model.summary())
results = lstm_model.evaluate(X_train_lstm, y_train_lstm, batch_size=128)
print("Train loss: ", results)
results = lstm_model.evaluate(X_val_lstm, y_val_lstm, batch_size=128)
print("Validation loss: ", results)
results = lstm_model.evaluate(X_test_lstm, y_test_lstm, batch_size=128)
print("Test loss: ", results)

##################
# Save the model #
##################
lstm_model_path = os.path.join(out_path, 'lstm-ball-bounce', 'my_lstm_model.keras')
lstm_model.save(lstm_model_path)

# You can also load it for other workflows
# new_lstm_model = tf.keras.models.load_model(lstm_model_path)

# Predict Data

Now that the model has been created, we can go ahead and predict our values. Since we only predict one step at a time, we need to keep predicting with a new window containing the newest prediction.

In [ ]:
# Predict rest of series based on initial window:
def prediction(X_predict, init_time, tot_time):
    
    X_predict_temp = np.array([X_predict[0][:window]])
    
    print(X_predict_temp)

    X_whole = X_predict_temp
    
    n_loops = int(tot_time-init_time )
    for i in range(n_loops):

        print(f'Predicting time step: {init_time+i+1} of {tot_time}')
        X_predict = lstm_model.predict(X_predict_temp)

        # Concatenating to whole guess
        X_whole=np.concatenate((X_whole,[X_predict]), axis=1)

        # New prediction window with latest prediction
        X_predict_temp = X_whole[:,-window:,:]

    return X_whole

if data_type == 'cyclic':

    init_time_train = window
    X_whole_train = prediction(X_train_lstm_for_predict, init_time_train, X.shape[0])
    X_whole_train_unscaled = scaler.inverse_transform(X_whole_train[0])

    init_time_val =  window+len(X_train_temp['x_pos'])
    X_whole_val = prediction(X_val_lstm_for_predict,init_time_val, X.shape[0])
    X_whole_val_unscaled = scaler.inverse_transform(X_whole_val[0])

    init_time_test =  window+len(X_train_temp['x_pos'])+len(X_val_temp['x_pos'])
    X_whole_test = prediction(X_test_lstm_for_predict,init_time_test, X.shape[0])
    X_whole_test_unscaled = scaler.inverse_transform(X_whole_test[0])

    #######################
    # Plot the prediction #
    #######################

    fig, ax = plt.subplots(nrows=3,sharex=True,figsize=(9,9))
    fig.suptitle('Whole Prediction')
    pos_train = scaler.inverse_transform(X_train_lstm_for_predict[0])
    pos_val = scaler.inverse_transform(X_val_lstm_for_predict[0])
    pos_test = scaler.inverse_transform(X_test_lstm_for_predict[0]) 
    
    x_pos = list(pos_train[:,0]) +  list(pos_val[:,0]) +  list(pos_test[:,0])
    ax[0].plot(time,x_pos, label='Original')
    ax[0].plot(time[:window],X_whole_train_unscaled [:window,0], label='Train Initial Guess')
    ax[0].plot(time[window-1:],X_whole_train_unscaled [window-1:,0], label='Train Whole Prediction')
    ax[0].plot(time_val[:window],X_whole_val_unscaled [:window,0], label='Validation Initial Guess')
    ax[0].plot(time[init_time_val-1:],X_whole_val_unscaled [window-1:,0], label='Validation Whole Prediction')
    ax[0].plot(time_test[:window],X_whole_test_unscaled [:window,0], label='Test Initial Guess')
    ax[0].plot(time[init_time_test-1:],X_whole_test_unscaled [window-1:,0], label='Test Whole Prediction')
    ax[0].legend(fontsize='xx-small')
    ax[0].set_title('x_pos')

    y_pos = list(pos_train[:,1]) +  list(pos_val[:,1]) +  list(pos_test[:,1])
    ax[1].plot(time,y_pos, label='Original')
    ax[1].plot(time[:window],X_whole_train_unscaled [:window,1], label='Train Initial Guess')
    ax[1].plot(time[window-1:],X_whole_train_unscaled [window-1:,1], label='Train Whole Prediction')
    ax[1].plot(time_val[:window],X_whole_val_unscaled [:window,1], label='Validation Initial Guess')
    ax[1].plot(time[init_time_val-1:],X_whole_val_unscaled [window-1:,1], label='Validation Whole Prediction')
    ax[1].plot(time_test[:window],X_whole_test_unscaled [:window,1], label='Test Initial Guess')
    ax[1].plot(time[init_time_test-1:],X_whole_test_unscaled [window-1:,1], label='Test Whole Prediction')
    ax[1].legend(fontsize='xx-small')
    ax[1].set_title('y_pos')

    z_pos = list(pos_train[:,2]) +  list(pos_val[:,2]) +  list(pos_test[:,2])
    ax[2].plot(time,z_pos, label='Original')
    ax[2].plot(time[:window],X_whole_train_unscaled [:window,2], label='Train Initial Guess')
    ax[2].plot(time[window-1:],X_whole_train_unscaled [window-1:,2], label='Train Whole Prediction')
    ax[2].plot(time_val[:window],X_whole_val_unscaled [:window,2], label='Validation Initial Guess')
    ax[2].plot(time[init_time_val-1:],X_whole_val_unscaled [window-1:,2], label='Validation Whole Prediction')
    ax[2].plot(time_test[:window],X_whole_test_unscaled [:window,2], label='Test Initial Guess')
    ax[2].plot(time[init_time_test-1:],X_whole_test_unscaled [window-1:,2], label='Test Whole Prediction')
    ax[2].legend(fontsize='xx-small')
    ax[2].set_title('z_pos')

    plt.tight_layout()
    fig.savefig(os.path.join(image_path, 'cyclic_whole_prediction.png'))
    
elif data_type == 'acyclic':

    X_whole_train = prediction(X_train_lstm_for_predict, window, X.shape[0])
    X_whole_train_unscaled = scaler.inverse_transform(X_whole_train[0])

    X_whole_val = prediction(X_val_lstm_for_predict,window, X.shape[0])
    X_whole_val_unscaled = scaler.inverse_transform(X_whole_val[0])

    X_whole_test = prediction(X_test_lstm_for_predict,window, X.shape[0])
    X_whole_test_unscaled = scaler.inverse_transform(X_whole_test[0])

    #######################
    # Plot the prediction #
    #######################

    fig, ax = plt.subplots(nrows=3,ncols=3,sharex=True,figsize=(9,9))
    fig.suptitle('Whole Prediction')

    # x_pos
    ax[0,0].plot(time,scaler.inverse_transform(X_train_lstm_for_predict[0])[:,0], label='Original')
    ax[0,0].plot(time[:window],X_whole_train_unscaled [:window,0], label='Initial Guess')
    ax[0,0].plot(time[window-1:],X_whole_train_unscaled [window-1:,0], label='Whole Prediction')
    ax[0,0].legend(fontsize='xx-small')
    ax[0,0].set_title('Train x_pos')

    ax[0,1].plot(time,scaler.inverse_transform(X_val_lstm_for_predict[0])[:,0], label='Original')
    ax[0,1].plot(time[:window],X_whole_val_unscaled [:window,0], label='Initial Guess')
    ax[0,1].plot(time[window-1:],X_whole_val_unscaled [window-1:,0], label='Whole Prediction')
    ax[0,1].legend(fontsize='xx-small')
    ax[0,1].set_title('Validation x_pos')

    ax[0,2].plot(time,scaler.inverse_transform(X_test_lstm_for_predict[0])[:,0], label='Original')
    ax[0,2].plot(time[:window],X_whole_test_unscaled [:window,0], label='Initial Guess')
    ax[0,2].plot(time[window-1:],X_whole_test_unscaled [window-1:,0], label='Whole Prediction')
    ax[0,2].legend(fontsize='xx-small')
    ax[0,2].set_title('Test x_pos')

    # y_pos
    ax[1,0].plot(time,scaler.inverse_transform(X_train_lstm_for_predict[0])[:,1], label='Original')
    ax[1,0].plot(time[:window],X_whole_train_unscaled [:window,1], label='Initial Guess')
    ax[1,0].plot(time[window-1:],X_whole_train_unscaled [window-1:,1], label='Whole Prediction')
    ax[1,0].legend(fontsize='xx-small')
    ax[1,0].set_title('Train y_pos')

    ax[1,1].plot(time,scaler.inverse_transform(X_val_lstm_for_predict[0])[:,1], label='Original')
    ax[1,1].plot(time[:window],X_whole_val_unscaled [:window,1], label='Initial Guess')
    ax[1,1].plot(time[window-1:],X_whole_val_unscaled [window-1:,1], label='Whole Prediction')
    ax[1,1].legend(fontsize='xx-small')
    ax[1,1].set_title('Validation y_pos')


    ax[1,2].plot(time,scaler.inverse_transform(X_test_lstm_for_predict[0])[:,1], label='Original')
    ax[1,2].plot(time[:window],X_whole_test_unscaled [:window,1], label='Initial Guess')
    ax[1,2].plot(time[window-1:],X_whole_test_unscaled [window-1:,1], label='Whole Prediction')
    ax[1,2].legend(fontsize='xx-small')
    ax[1,2].set_title('Test y_pos')

    # y_pos
    ax[2,0].plot(time,scaler.inverse_transform(X_train_lstm_for_predict[0])[:,2], label='Original')
    ax[2,0].plot(time[:window],X_whole_train_unscaled [:window,2], label='Initial Guess')
    ax[2,0].plot(time[window-1:],X_whole_train_unscaled [window-1:,2], label='Whole Prediction')
    ax[2,0].legend(fontsize='xx-small')
    ax[2,0].set_title('Train z_pos')

    ax[2,1].plot(time,scaler.inverse_transform(X_val_lstm_for_predict[0])[:,2], label='Original')
    ax[2,1].plot(time[:window],X_whole_val_unscaled [:window,2], label='Initial Guess')
    ax[2,1].plot(time[window-1:],X_whole_val_unscaled [window-1:,2], label='Whole Prediction')
    ax[2,1].legend(fontsize='xx-small')
    ax[2,1].set_title('Validation z_pos')


    ax[2,2].plot(time,scaler.inverse_transform(X_test_lstm_for_predict[0])[:,2], label='Original')
    ax[2,2].plot(time[:window],X_whole_test_unscaled [:window,2], label='Initial Guess')
    ax[2,2].plot(time[window-1:],X_whole_test_unscaled [window-1:,2], label='Whole Prediction')
    ax[2,2].legend(fontsize='xx-small')
    ax[2,2].set_title('Test z_pos')

    plt.tight_layout()
    fig.savefig(os.path.join(image_path, 'acyclic_whole_prediction.png'))